In [62]:
import pandas as pd
import csv

dataset=[line.rstrip() for line in open ('C:\\Users\\ggovinda\\Downloads\\dataset.csv')]
dataset = pd.read_csv("C:\\Users\\ggovinda\\Downloads\\dataset.csv",sep='\t',quoting=csv.QUOTE_NONE, names=['label','message'])

print(dataset.shape)
print(dataset.head(3))

data_col=list(dataset.columns)
print(data_col)

print(dataset.groupby('label').describe())
dataset.head(3)



(5574, 2)
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
['label', 'message']
      message                                                               
        count unique                                                top freq
label                                                                       
ham      4827   4518                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [65]:
dataset_target = dataset["label"]
print(dataset_target)

0        ham
1        ham
2       spam
3        ham
4        ham
5       spam
6        ham
7        ham
8       spam
9       spam
10       ham
11      spam
12      spam
13       ham
14       ham
15      spam
16       ham
17       ham
18       ham
19      spam
20       ham
21       ham
22       ham
23       ham
24       ham
25       ham
26       ham
27       ham
28       ham
29       ham
        ... 
5544     ham
5545     ham
5546     ham
5547     ham
5548     ham
5549    spam
5550     ham
5551     ham
5552     ham
5553     ham
5554     ham
5555     ham
5556     ham
5557     ham
5558     ham
5559     ham
5560     ham
5561     ham
5562     ham
5563     ham
5564     ham
5565     ham
5566     ham
5567     ham
5568    spam
5569    spam
5570     ham
5571     ham
5572     ham
5573     ham
Name: label, Length: 5574, dtype: object


In [83]:
import nltk
from nltk.tokenize import word_tokenize

def split_token(message):
    message = message.lower()
   # message = unicode(message,'utf8')
    #message = message.encode('utf-8')
    word_token=word_tokenize(message)
    return word_token

dataset["token_word"]=dataset.apply(lambda row:split_token(row['message']), axis=1)

    

In [84]:
from nltk.stem.wordnet import WordNetLemmatizer
def split_lemma(message):
    lemma = []
    lemmatizer = WordNetLemmatizer()
    for word in message:
        a=lemmatizer.lemmatize(word)
        lemma.append(a)
    return lemma

dataset['lemma_word']=dataset.apply(lambda row:split_lemma(row['token_word']), axis=1)
print("token_word: ", dataset['token_word'],[11])
print("lemma_word: ", dataset['lemma_word'],[11])

token_word:  0       [go, until, jurong, point, ,, crazy.., availab...
1                [ok, lar, ..., joking, wif, u, oni, ...]
2       [free, entry, in, 2, a, wkly, comp, to, win, f...
3       [u, dun, say, so, early, hor, ..., u, c, alrea...
4       [nah, i, do, n't, think, he, goes, to, usf, ,,...
5       [freemsg, hey, there, darling, it, 's, been, 3...
6       [even, my, brother, is, not, like, to, speak, ...
7       [as, per, your, request, 'melle, melle, (, oru...
8       [winner, !, !, as, a, valued, network, custome...
9       [had, your, mobile, 11, months, or, more, ?, u...
10      [i, 'm, gon, na, be, home, soon, and, i, do, n...
11      [six, chances, to, win, cash, !, from, 100, to...
12      [urgent, !, you, have, won, a, 1, week, free, ...
13      [i, 've, been, searching, for, the, right, wor...
14       [i, have, a, date, on, sunday, with, will, !, !]
15      [xxxmobilemovieclub, :, to, use, your, credit,...
16              [oh, k, ..., i, 'm, watching, here, :, )]
1

In [85]:
from nltk.corpus import stopwords

def split_stop_words(message):
    stop_words_split = []
    stop_words = set(stopwords.words('english'))
    stop_words_split = ' '.join([word for word in message if word not in stop_words])
    return stop_words_split

dataset["stop_words"]=dataset.apply(lambda row:split_stop_words(row['lemma_word']), axis=1)

Training_data = pd.Series(list(dataset["stop_words"]))
Training_label = pd.Series(list(dataset["label"]))

In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tdm_vector = CountVectorizer(ngram_range=(1,2), min_df=(1/len(Training_label)), max_df=0.7)
total_text = tdm_vector.fit(Training_data)
tdm_total = total_text.transform(Training_data)

tfidf_vector = TfidfVectorizer(ngram_range=(1,2), min_df=(1/len(Training_label)), max_df=0.7)
tfidf_text = tfidf_vector.fit(Training_data)
tfidf_total = tfidf_text.transform(Training_data)

In [109]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(tfidf_total, Training_label, test_size=.1)


In [110]:
from sklearn.tree import DecisionTreeClassifier

classifier=DecisionTreeClassifier()
classifier = classifier.fit(train_data, train_label)
message_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)

print('Decision Tree Classifier : ',score)

Decision Tree Classifier :  0.982078853046595


In [111]:
from sklearn.linear_model import SGDClassifier

seed=7
classifier = SGDClassifier(loss='modified_huber', shuffle=True, random_state=seed)
classifier = classifier.fit(train_data, train_label)
score_sgd = classifier.score(test_data, test_label)
print("SGD score: ",score_sgd )


SGD score:  0.982078853046595


c:\users\ggovinda\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [118]:
from sklearn.svm import SVC
seed=7
classifier = SVC(C=0.025, random_state=seed)
classifier = classifier.fit(train_data, train_label)
score_svc = classifier.score(test_data, test_label)
print("SVC score : ", score_svc)

c:\users\ggovinda\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC score :  0.8637992831541219


In [119]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score = classifier.score(test_data, test_label)

print('Random Forest Classifier : ',score)

Random Forest Classifier :  0.8637992831541219


In [120]:

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=5, n_estimators=15, max_features=60,random_state=seed)

classifier = classifier.fit(train_data, train_label)

score=classifier.score(test_data, test_label)

print('Random Forest classification after model tuning',score)

Random Forest classification after model tuning 0.8637992831541219


In [126]:
seed=7

from sklearn.model_selection import StratifiedShuffleSplit
#creating cross validation object with 10% test size 
cross_val = StratifiedShuffleSplit(Training_label,test_size=0.1,random_state=seed)

In [130]:
classifiers = [
    DecisionTreeClassifier(),
    SGDClassifier(loss='modified_huber', shuffle=True),
    SVC(C=0.025),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10),
   ]

for clf in classifiers:
    score=0
    for train_index, test_index in cross_val:
        X_train, X_test = tfidf_total['train_index'], tfidf_total['test_index']
        y_train, y_test = Training_label['train_index'], Training_label['test_index']
        clf.fit(X_train, y_train)
        score=score+clf.score(X_test,y_test)
    print (score)
    
        

TypeError: 'StratifiedShuffleSplit' object is not iterable

In [137]:
from sklearn.metrics import accuracy_score
print('Accuracy Score',accuracy_score(test_label,message_predicted_target))  
classifier = classifier.fit(train_data, train_label)
clasifier = classifier.score(test_data, test_label)
test_label.value_counts()

Accuracy Score 0.982078853046595


ham     482
spam     76
dtype: int64

In [138]:
from sklearn.metrics import confusion_matrix

print('Confusion Matrix',confusion_matrix(test_label,message_predicted_target))

Confusion Matrix [[479   3]
 [  7  69]]


In [139]:
from sklearn.metrics import classification_report

target_names = ['spam', 'ham']

print(classification_report(test_label, message_predicted_target, target_names=target_names))

              precision    recall  f1-score   support

        spam       0.99      0.99      0.99       482
         ham       0.96      0.91      0.93        76

   micro avg       0.98      0.98      0.98       558
   macro avg       0.97      0.95      0.96       558
weighted avg       0.98      0.98      0.98       558



In [140]:
print(dataset['label'].unique())

['ham' 'spam']
